In [ ]:
# Import necessary libraries
from flask import Flask, request, render_template
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

# Create a Flask application
app = Flask(__name__, template_folder='.')

# Initialize a TfidfVectorizer
vectorizer = TfidfVectorizer()

# Store the texts in a global variable
texts = []

# Define the route for the home page
@app.route('/')
def home():
    # Render the HTML template
    return render_template('template.html')

# Define the route for uploading a file
@app.route('/upload_file', methods=['POST'])
def upload_file():
    # Get the file from the request
    file = request.files['file']

    # Read the file and split it into lines
    global texts
    texts = file.read().decode('utf-8').split('\n')

    # Return a success message
    return {'status': 'File uploaded successfully'}

# Define the route for getting the news
@app.route('/get_news', methods=['GET'])
def get_news():
    # Return the news
    global texts
    return {'news': texts}

# Define the route for processing the text
@app.route('/process_text', methods=['POST'])
def process_text():
    # Get the selected article index from the request
    selected_article_index = int(request.get_json()['selected_article_index'])

    # Convert the texts to a TF-IDF matrix
    global texts
    tfidf_matrix = vectorizer.fit_transform(texts)

    # Compute the cosine similarities between the selected article and all articles
    cosine_similarities = cosine_similarity(tfidf_matrix[selected_article_index:selected_article_index+1], tfidf_matrix).flatten()

    # Set the similarity of the selected article to -1 so it won't be recommended
    cosine_similarities[selected_article_index] = -1

    # Find the index of the most similar article
    related_docs_indices = cosine_similarities.argsort()[:-2:-1]

    # Get the recommended article
    recommended_article = texts[related_docs_indices[0]]

    # Return the selected and recommended articles
    return {'selected_article': texts[selected_article_index], 'recommended_article': recommended_article}

# Run the Flask application
if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Jan/2024 10:15:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 10:15:09] "POST /upload_file HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 10:15:10] "GET /get_news HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 10:15:14] "POST /process_text HTTP/1.1" 200 -
